In [1]:
# https://www.youtube.com/watch?v=f-JCCOHwx1c&list=PLZoTAELRMXVPGU70ZGsckrMdr0FteeRUi&index=52

In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
batch_size=64 
epochs=100 
latent_dim=256
num_samples=10000
data_path= '../sourcefile/fra.txt'

In [ ]:
## vectorize the data
## input and counting character
input_texts= [] 
target_texts= []
input_charaters = set()
target_charaters = set() 
with open(data_path,'r', encoding='utf-8') as f: 
    lines=f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]: 
    input_text, target_text, _=line.split('\t')
    target_text='\t'+ target_text+'\n' 
    input_texts.append(input_text) 
    target_texts.append(target_text) 
    for char in input_text: 
        if char not in input_charaters: 
            input_charaters.add(char)
    for char in target_text:
        if char not in target_charaters:
            target_charaters.add(char)

In [4]:
len(input_charaters)

70

In [5]:
input_charaters=sorted(list(input_charaters)) 
target_charaters=sorted(list(target_charaters)) 
num_encoder_tokens=len(input_charaters) 
num_decoder_tokens=len(target_charaters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts])

In [6]:
print('Number of sample :', len(input_texts))
print('Number of Unique input Token', num_encoder_tokens)
print('Number of Unique output Token', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length) 
print('Max sequence length for output:', max_decoder_seq_length)


Number of sample : 10000
Number of Unique input Token 70
Number of Unique output Token 91
Max sequence length for inputs: 14
Max sequence length for output: 59


In [7]:
input_token_index=dict( 
    [(char, i) for i, char in enumerate(input_charaters)])
target_token_index= dict( 
    [(char, i) for i, char in enumerate(target_charaters)]) 

In [8]:
encoder_input_data=np.zeros( 
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), 
    dtype='float32')
decoder_input_data=np.zeros( 
    (len(input_texts), max_decoder_seq_length,num_decoder_tokens), 
    dtype='float32') 
decoder_target_data=np.zeros( 
    (len(input_texts), max_decoder_seq_length,num_decoder_tokens), 
    dtype='float32')


In [9]:
for i, (input_text, target_text) in enumerate(zip(input_texts,target_texts)): 
    for t, char in enumerate(input_text): 
        encoder_input_data[i,t,input_token_index[char]]=1.
    encoder_input_data[i,t+1:, input_token_index[' ']]=1. 
    for t, char in enumerate(target_text): 
        decoder_input_data[i,t, target_token_index[char]]=1. 
        if t> 0: 
            decoder_input_data[i,t-1:, target_token_index[char]]=1.
    decoder_input_data[i,t+1:, target_token_index[' ']] =1.
    decoder_target_data[i,t:, target_token_index[' ']]=1.

In [10]:
encoder_input_data[0].shape

(14, 70)

In [14]:
# encoder_inpts=Input(shape=(None,num_decoder_tokens)) the line has been fixed 
# https://gemini.google.com/app/e874598beb5a709a  solution link
#https://keras.io/examples/nlp/lstm_seq2seq/ blog soluton link
encoder_inpts = Input(shape=(None, num_encoder_tokens))
encoder=LSTM(latent_dim, return_state=True) 
encoder_outputs, state_h, state_c=encoder(encoder_inpts) 
encoder_states=[state_h,state_c]

In [15]:
## Define an input sequences and process  
decoder_inputs=Input(shape=(None,num_decoder_tokens)) 
decoder_lstm=LSTM(latent_dim, return_sequences=True, return_state=True) 
decoder_outputs,_,_= decoder_lstm(decoder_inputs, initial_state=encoder_states) 
decoder_dense=Dense(num_decoder_tokens, activation='softmax') 
decoder_outputs=decoder_dense(decoder_outputs) 

In [ ]:
# model=Model([encoder_inpts,decoder_inputs], decoder_outputs)
# model.compile(optimizer='rmsprop', loss='categorocal_crossentropy', 
#               metrics=['accuracy']) 
# model.fit=([encoder_input_data,decoder_input_data], decoder_target_data, 
#             batch_size=64, epochs=10, validation_split=0.2)


model = Model([encoder_inpts, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 57s 424ms/step - accuracy: 0.6737 - loss: 0.2705 - val_accuracy: 0.6649 - val_loss: 0.0258
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 365ms/step - accuracy: 0.7029 - loss: 0.0557 - val_accuracy: 0.6649 - val_loss: 0.0259
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 51s 411ms/step - accuracy: 0.7016 - loss: 0.0563 - val_accuracy: 0.6649 - val_loss: 0.0277
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 362ms/step - accuracy: 0.7024 - loss: 0.0504 - val_accuracy: 0.6649 - val_loss: 0.0369
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 58s 461ms/step - accuracy: 0.7015 - loss: 0.0442 - val_accuracy: 0.6649 - val_loss: 0.0432
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 52s 418ms/step - accuracy: 0.7041 - loss: 0.0404 - val_accuracy: 0.6649 - val_loss: 0.0249
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 86s 449ms/step - accuracy: 0.7040 - loss: 0.0314 - val_accuracy: 0.6649 - val_loss: 0.0240
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 57s 456ms/step - accuracy: 0.7024 -

In [ ]:
# https://www.youtube.com/watch?v=f-JCCOHwx1c&list=PLZoTAELRMXVPGU70ZGsckrMdr0FteeRUi&index=52